In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
%matplotlib inline
import matplotlib.pyplot as plt
import re
import datetime

from scipy import stats
from scipy.stats import pearsonr, spearmanr

from googletrans import Translator

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv('daily8_5.csv')
data['dateline'] = pd.to_datetime(data.dateline)
df = data.copy()
df = df[df.text_label != 'False']
df2 = df

In [20]:
df = data.copy()
df = df[df.text_label != 'False']
df2 = df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237706 entries, 1 to 331869
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   index           237706 non-null  int64         
 1   id              237706 non-null  object        
 2   url             237706 non-null  object        
 3   headline        237706 non-null  object        
 4   dateline        237706 non-null  datetime64[ns]
 5   text            237706 non-null  object        
 6   tags            237706 non-null  object        
 7   source          237706 non-null  object        
 8   text_label      237706 non-null  object        
 9   text_score      237706 non-null  object        
 10  headline_label  237706 non-null  object        
 11  headline_score  237706 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 23.6+ MB


In [21]:
transcription_table=pd.DataFrame(
    {
        'source': ['Alittihad','Echoroukonline','Ryiadh','SaudiYoum','Techreen', 'Alqabas', 'Almustaqbal','Almasryalyoum', 'Youm7','Sabanews'],
        'country': ['UAE','Algeria','Saudi_Arabia','Saudi_Arabia','Syria', 'Kuwait', 'Lebanon','Egypt','Egypt','Yemen']
    }
)
mapping = transcription_table.set_index('source').to_dict()['country']
df['country'] = df['source'].apply(lambda x: mapping.get(x))

In [22]:
## Functions possibly required for this section

def within_30_days(df_dateline, date):
    
    if (df_dateline - date).days < 30 and (df_dateline - date).days > -30:
        return 1
    else:
        return 0
    
important_dates = {
    'september_11th': pd.to_datetime('09-11-2001'),
    #'capture_of_baghdad': pd.to_datetime('04-09-2003'),
    #'nick_berg': pd.to_datetime('05-12-2004'),
    #'iran_nulcear': pd.to_datetime('08-30-2006'),
    #'arab_spring': pd.to_datetime('12-20-2011')
}

for event, date in important_dates.items():
    df['sept_11'] = df.dateline.apply(within_30_days, args = (date,))
    
    
def within_30_days(df_dateline, date):
    
    if (df_dateline - date).days < 30 and (df_dateline - date).days > -30:
        return 1
    else:
        return 0
    
important_dates = {
    #'september_11th': pd.to_datetime('09-11-2001'),
    'capture_of_baghdad': pd.to_datetime('04-09-2003'),
    #'nick_berg': pd.to_datetime('05-12-2004'),
    #'iran_nulcear': pd.to_datetime('08-30-2006'),
    #'arab_spring': pd.to_datetime('12-20-2011')
}

for event, date in important_dates.items():
    df['capt_bag'] = df.dateline.apply(within_30_days, args = (date,))
    
    
    
    
def within_30_days(df_dateline, date):
    
    if (df_dateline - date).days < 30 and (df_dateline - date).days > -30:
        return 1
    else:
        return 0
    
important_dates = {
    #'september_11th': pd.to_datetime('09-11-2001'),
    #'capture_of_baghdad': pd.to_datetime('04-09-2003'),
    'nick_berg': pd.to_datetime('05-12-2004'),
    #'iran_nulcear': pd.to_datetime('08-30-2006'),
    #'arab_spring': pd.to_datetime('12-20-2011')
}

for event, date in important_dates.items():
    df['nick'] = df.dateline.apply(within_30_days, args = (date,))
    
    
    
def within_30_days(df_dateline, date):
    
    if (df_dateline - date).days < 30 and (df_dateline - date).days > -30:
        return 1
    else:
        return 0
    
important_dates = {
    #'september_11th': pd.to_datetime('09-11-2001'),
    #'capture_of_baghdad': pd.to_datetime('04-09-2003'),
    #'nick_berg': pd.to_datetime('05-12-2004'),
    #'iran_nulcear': pd.to_datetime('08-30-2006'),
    'arab_spring': pd.to_datetime('12-20-2011')
}

for event, date in important_dates.items():
    df['arab'] = df.dateline.apply(within_30_days, args = (date,))

In [23]:
topics = {'America' : 'أمريكا',
            'American' : 'أمريكيّ',
            'American (f)' : 'أمريكيّة',
            'American (pl)' : 'أمريكيّين',
            'The United States' : 'الولايات المتحدة',
            'The United States' : 'دول موحّدة',
            'Washington' : 'واشنطن',
            'Bush' : 'بوش',
            'Obama' : 'أوباما',
            'Cheney' : 'تشيني',
            'Clinton' : 'كلينتون',
            'Osama Bin Laden' : 'أسامة بن لادن',
            'Al Gore' : 'آل غور',
            'World Trade Center' : 'مركز التجارة العالمي',
            '9/11' : '9/11',
            'September 11' : '11 سبتمبر',
            'Gulf War' : 'حرب الخليج',
            'Google' : 'غوغل',
            'Facebook' : 'فيسبوك',
            'Al Qaida' : 'القاعدة'}

def flip_key_value_pairs(dicts):
    
    res = dict((v,k) for k,v in dicts.items())    
    return res


def make_eng_tags(df_tags):
    eng_tags =[]
    
    rev_topics = flip_key_value_pairs(topics)
    for key, value in rev_topics.items():
        if key in df_tags:
            eng_tags.append(value)
            
    return eng_tags

df['english_tags'] = df.tags.apply(make_eng_tags)
#translates tags to english

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237706 entries, 1 to 331869
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   index           237706 non-null  int64         
 1   id              237706 non-null  object        
 2   url             237706 non-null  object        
 3   headline        237706 non-null  object        
 4   dateline        237706 non-null  datetime64[ns]
 5   text            237706 non-null  object        
 6   tags            237706 non-null  object        
 7   source          237706 non-null  object        
 8   text_label      237706 non-null  object        
 9   text_score      237706 non-null  object        
 10  headline_label  237706 non-null  object        
 11  headline_score  237706 non-null  float64       
 12  country         237706 non-null  object        
 13  sept_11         237706 non-null  int64         
 14  capt_bag        237706 non-null  int

In [25]:
df['english_tags'] = df.english_tags.astype('str')

In [26]:
df["quarter"] = pd.PeriodIndex(df.dateline, freq='Q')
df["year"] = pd.PeriodIndex(df.dateline, freq='Y')
df["half_year"] = pd.PeriodIndex(df.dateline, freq='6M')

In [28]:
df['year'] = df['year'].astype('str')
df['quarter'] = df['quarter'].astype('str')
df['half_year'] = df['half_year'].astype('str')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237706 entries, 1 to 331869
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   index           237706 non-null  int64         
 1   id              237706 non-null  object        
 2   url             237706 non-null  object        
 3   headline        237706 non-null  object        
 4   dateline        237706 non-null  datetime64[ns]
 5   text            237706 non-null  object        
 6   tags            237706 non-null  object        
 7   source          237706 non-null  object        
 8   text_label      237706 non-null  object        
 9   text_score      237706 non-null  object        
 10  headline_label  237706 non-null  object        
 11  headline_score  237706 non-null  float64       
 12  country         237706 non-null  object        
 13  sept_11         237706 non-null  int64         
 14  capt_bag        237706 non-null  int

In [31]:
x = pd.get_dummies(df[['source', 'country', 'quarter', 'year', 
                    'half_year']])

In [32]:
def encode_tags(df):
    list_of_tags = []
    for tag in df.english_tags.values:
        list_of_tags.extend([val[1:-1] for val in tag[1:-1].split(', ')])
    tag_list = list(set(list_of_tags))
    for tag in tag_list:
        df[tag] = 0
    for i, tag in enumerate(df.english_tags):
        for t in tag_list:
            if t in tag:
                df[t].iloc[i] = 1
    return df

In [33]:
encode_tags(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237706 entries, 1 to 331869
Data columns (total 39 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               237706 non-null  int64         
 1   id                  237706 non-null  object        
 2   url                 237706 non-null  object        
 3   headline            237706 non-null  object        
 4   dateline            237706 non-null  datetime64[ns]
 5   text                237706 non-null  object        
 6   tags                237706 non-null  object        
 7   source              237706 non-null  object        
 8   text_label          237706 non-null  object        
 9   text_score          237706 non-null  object        
 10  headline_label      237706 non-null  object        
 11  headline_score      237706 non-null  float64       
 12  country             237706 non-null  object        
 13  sept_11             237706 no

In [36]:
#All columns that need to be dropped
x2 = df.drop(columns=['index', 'id', 'url', 'headline', 'dateline', 'text', 'english_tags', 'tags', 'text_label', 'text_score', 'headline_label', 'headline_score', 'source', 'country', 'quarter', 'year', 
                    'half_year'])

In [37]:
x2

,sept_11,capt_bag,nick,arab,Al Qaida,Facebook,Al Gore,Google,Washington,Osama Bin Laden,...,September 11,Obama,America,Gulf War,9/11,American,American (pl),Clinton,World Trade Center,Bush
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331864,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
331866,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [38]:
x3 = pd.concat([x, x2], axis=1)

In [39]:
x3

,source_Alittihad,source_Almasryalyoum,source_Almustaqbal,source_Alqabas,source_Echoroukonline,source_Ryiadh,source_Sabanews,source_SaudiYoum,source_Techreen,source_Youm7,...,September 11,Obama,America,Gulf War,9/11,American,American (pl),Clinton,World Trade Center,Bush
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331864,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331865,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
331866,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331867,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [42]:
df['text_label'] = df['text_label'].replace('neutral', 0)
df['text_label'] = df['text_label'].replace('positive', 1)
df['text_label'] = df['text_label'].replace('negative', -1)
df['text_label'] = df['text_label'].astype(int)
y = df['text_label']

In [43]:
f_selector = SelectKBest(f_regression, k=8)

# find the top 8 X's correlated with y
f_selector.fit(x3, y)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = x3.iloc[:,feature_mask].columns.tolist()
f_feature

['source_Alittihad',
 'source_Almasryalyoum',
 'source_Techreen',
 'source_Youm7',
 'country_Syria',
 'country_UAE',
 'America',
 'Bush']

In [48]:
scores = f_selector.scores_


# get the ranks
var_ranks = f_selector.scores_
# get the variable names
var_names = x3.columns.tolist()
# combine ranks and names into a df for clean viewing
rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Scores': var_ranks})
# sort the df by rank
scores = rfe_ranks_df.sort_values('Scores')

In [57]:
scores.sort_values(by='Scores', ascending=False).head(30)

,Var,Scores
8,source_Techreen,8943.527952
15,country_Syria,8943.527952
1,source_Almasryalyoum,2355.047357
260,America,1749.399832
0,source_Alittihad,1254.152884
16,country_UAE,1254.152884
9,source_Youm7,1254.076024
267,Bush,825.488801
7,source_SaudiYoum,563.408020
74,year_2001,537.509658
